In [1]:
import pandas as pd
import math
import os.path
import time
import numpy as np
import talib
from talib import abstract
from binance.client import Client
from datetime import timedelta, datetime
from dateutil import parser
#backtest lib
from backtesting import Backtest
from backtesting.lib import SignalStrategy
from backtesting.lib import resample_apply


C:\ProgramData\Anaconda3\lib\site-packages\backtesting\_plotting.py:45: UserWarning: Jupyter Notebook detected. Setting Bokeh output to notebook. This may not work in Jupyter clients without JavaScript support (e.g. PyCharm, Spyder IDE). Reset with `backtesting.set_bokeh_output(notebook=False)`.
  warnings.warn('Jupyter Notebook detected. '


Loading BokehJS ...

In [2]:
### API
binance_api_key = 'TESZdWxapwvZG90v1SJ3zMahcXySUkYv3bxRRKXKkfCMANtUS0Dw38iWALACQuHU'    #Enter your own API-key here
binance_api_secret = 'Uw1V3mbqoXDm9II7tC8lHRrYcGslHKhqo2bEXGelt08bnAn5lk4EbYfbKmHjWguY' #Enter your own API-secret here
### CONSTANTS ----add 15min
binsizes = {"1m": 1, "5m": 5,"15m":15, "1h": 60, "4h": 240, "1d": 1440}
batch_size = 750
# api instance
binance_client = Client(api_key=binance_api_key, api_secret=binance_api_secret)

In [3]:
### get data funcs
def minutes_of_new_data(symbol, kline_size, data, source):
    if len(data) > 0:  old = parser.parse(data["timestamp"].iloc[-1])
    elif source == "binance": old = datetime.strptime('1 Jan 2017', '%d %b %Y')
    elif source == "bitmex": old = bitmex_client.Trade.Trade_getBucketed(symbol=symbol, binSize=kline_size, count=1, reverse=False).result()[0][0]['timestamp']
    if source == "binance": new = pd.to_datetime(binance_client.get_klines(symbol=symbol, interval=kline_size)[-1][0], unit='ms')
    if source == "bitmex": new = bitmex_client.Trade.Trade_getBucketed(symbol=symbol, binSize=kline_size, count=1, reverse=True).result()[0][0]['timestamp']
    return old, new

def get_all_binance(symbol, kline_size, save = False):
    filename = '%s-%s-data.csv' % (symbol, kline_size)
    # jsaw edit: not to get data from API if aiready exist data
    if os.path.isfile(filename): 
        print("read exist")
        data_df = pd.read_csv(filename)
        return data_df
    else: 
        data_df = pd.DataFrame()
    oldest_point, newest_point = minutes_of_new_data(symbol, kline_size, data_df, source = "binance")
    delta_min = (newest_point - oldest_point).total_seconds()/60
    available_data = math.ceil(delta_min/binsizes[kline_size])
    if oldest_point == datetime.strptime('1 Jan 2017', '%d %b %Y'): 
        print('Downloading all available %s data for %s. Be patient..!' % (kline_size, symbol))
    else: 
        print('Downloading %d minutes of new data available for %s, i.e. %d instances of %s data.' % (delta_min, symbol, available_data, kline_size))
    klines = binance_client.get_historical_klines(symbol, kline_size, oldest_point.strftime("%d %b %Y %H:%M:%S"), newest_point.strftime("%d %b %Y %H:%M:%S"))
    data = pd.DataFrame(klines, columns = ['timestamp', 'open', 'high', 'low', 'close', 'volume', 'close_time', 'quote_av', 'trades', 'tb_base_av', 'tb_quote_av', 'ignore' ])
    data['timestamp'] = pd.to_datetime(data['timestamp'], unit='ms')
    if len(data_df) > 0:
        temp_df = pd.DataFrame(data)
        data_df = data_df.append(temp_df)
    else: 
        data_df = data
    data_df.set_index('timestamp', inplace=True)
    data_df = data.astype(float)
    if save: 
        data_df.to_csv(filename)
    print('All caught up..!')
    return data_df

In [4]:
#get data since 2017.7.1

df_4hr = get_all_binance('BTCUSDT', '4h',True)
df_1hr = get_all_binance('BTCUSDT', '1h',True)
df_15min = get_all_binance('BTCUSDT', '15m',True)
df_5min = get_all_binance('BTCUSDT', '5m', True)
#df_1min = get_all_binance('BTCUSDT', '1m',True)

read exist
read exist
read exist
read exist


In [5]:
df = df_15min

df.set_index('timestamp', inplace=True)  # 将object 类型的日期列，转换为普通索引
#print(df.index)

df.index = pd.DatetimeIndex(df.index)  # 转换为时间戳索引
#print(df.index)

In [85]:
class StrategyBoll(SignalStrategy):
    #optimize parameter
    
    #15min 24,20
    RSIperiod = 24
    RSICloseUpTrigger = 70
    RSICloseDownTrigger = 20
    RSIOpenTrigger = 50
    dynSize = 90
    BollPeriod = 35
    
    
    def init(self):
        super().init()
        
        #strategy construct
        
        # self.I ( strategy func, self.close, variable pass to strategy func) ==> calc strategy & show on plot 
        upper,middle,lower= resample_apply('4H',talib.BBANDS,self.data.Close,self.BollPeriod)
        self.boll_4HR_up = upper
        self.boll_4HR_mid = middle
        self.boll_4HR_low = lower
        
        upper,middle,lower= resample_apply('D',talib.BBANDS,self.data.Close,self.BollPeriod)
        self.boll_1D_up = upper
        self.boll_1D_mid = middle
        self.boll_1D_low = lower

       
    def log(self, show_sig = False):
         #position active
        if self.close_long_sig & self.long_sig :
            with open('log.txt','a') as logFile:
                    logFile.write(str(self.data.index[-1])+"****long conflict.\n")
        if self.close_short_sig & self.short_sig :
            with open('log.txt','a') as logFile:
                    logFile.write(str(self.data.index[-1])+"****short conflict.\n")
        if self.position:
            if self.close_long_sig & self.position.is_long:
                with open('log.txt','a') as logFile:
                    logFile.write(str(self.data.index[-1])+ "PL:"+str(self.position.pl_pct) + " close long .\n")
            elif self.close_short_sig & self.position.is_short:
                with open('log.txt','a') as logFile:
                    logFile.write(str(self.data.index[-1])+ "PL:"+str(self.position.pl_pct) + " close short .\n")
            # did nothing but get signal
            elif self.long_sig & show_sig:
                with open('log.txt','a') as logFile:
                    logFile.write(str(self.data.index[-1])+" ##long sig.\n")
            elif self.short_sig & show_sig:
                with open('log.txt','a') as logFil:
                    logFile.write(str(self.data.index[-1])+" ##short sig.\n")
            elif self.close_long_sig & show_sig:
                with open('log.txt','a') as logFile:
                    logFile.write(str(self.data.index[-1])+" ##close long sig.\n")
            elif self.close_short_sig & show_sig:
                with open('log.txt','a') as logFile:
                    logFile.write(str(self.data.index[-1])+" ##close short sig.\n")
        else:
            if self.long_sig:
                with open('log.txt','a') as logFile:
                    logFile.write(str(self.data.index[-1])+" long.\n")
            if self.short_sig:
                with open('log.txt','a') as logFile:
                    logFile.write(str(self.data.index[-1])+" short.\n")
            if self.close_long_sig & show_sig:
                with open('log.txt','a') as logFile:
                    logFile.write(str(self.data.index[-1])+" ##close long sig.\n")
            if self.close_short_sig & show_sig:
                with open('log.txt','a') as logFile:
                    logFile.write(str(self.data.index[-1])+" ##close short sig.\n")
    def dynLog(self):
        if self.close_long_sig & self.position.is_long:
            with open('log.txt','a') as logFile:
                    logFile.write(str(self.data.index[-1])+" close long\n")
        if self.close_short_sig & self.position.is_short:
            self.position.close()
        if self.long_sig:
             with open('log.txt','a') as logFile:
                    logFile.write(str(self.data.index[-1])+" long.\n")
        if self.short_sig:
            self.sell()
        ##sig
        if self.close_long_sig & (not self.position):
            with open('log.txt','a') as logFile:
                logFile.write(str(self.data.index[-1])+"## close longsig\n")
                
    def next(self):
        # split strategy for combine
        long_boll = (self.data.Low[-1] < self.boll_1D_low[-1]) & (self.data.Close[-1] > self.boll_1D_low[-1])
        
        short_boll = (self.data.High[-1] > self.boll_1D_up[-1]) & (self.data.Close[-1] < self.boll_1D_up[-1])
               #combine signal
        self.long_sig = long_boll
        self.close_long_sig = short_boll
        self.short_sig = False;
        self.close_short_sig = False;
        #position active
        
        #if self.position:
        #    #close long
        #    if self.close_long_sig & self.position.is_long:
        #        self.position.close()
        #    #close short
        #    elif self.close_short_sig & self.position.is_short:
        #        self.position.close()
        #no any position
        #else:
        #    if self.long_sig:
        #        self.buy()
        #    if self.short_sig:
        #        self.sell()
                
        if self.close_long_sig & self.position.is_long:
            self.position.close()
        if self.close_short_sig & self.position.is_short:
            self.position.close()
        if self.long_sig:
            self.buy(size = self.dynSize/100)
        if self.short_sig:
            self.sell()
        self.log()
        #self.dynLog()
        
        #if(self.position.pl_pct > 0.3):
        #   self.position.close()
        #if(self.position.pl_pct < -0.05):
        #    self.position.close()
            
        super().next()

In [66]:
class StrategyMACD(SignalStrategy):
    #optimize parameter
    
    #15min 24,20
    RSIperiod = 24
    RSICloseUpTrigger = 70
    RSICloseDownTrigger = 20
    RSIOpenTrigger = 50
    dynSize = 50
    BollPeriod = 35
    
    def init(self):
        super().init()
        
        #strategy construct
        
        # self.I ( strategy func, self.close, variable pass to strategy func) ==> calc strategy & show on plot 
        macdFast, macdSlow, histogram = self.I(talib.MACD,self.data.Close,fastperiod=12, slowperiod=26, signalperiod=9)
        self.ema5 = self.I(talib.EMA,self.data.Close,5)
        self.ema10 = self.I(talib.EMA,self.data.Close,10)
        self.ema20 = self.I(talib.EMA,self.data.Close,20)
        self.ema60 = self.I(talib.EMA,self.data.Close,60)
        self.ema200 = self.I(talib.EMA,self.data.Close,200)
        self.ma5 = self.I(talib.MA,self.data.Close,5)
        self.ma10 = self.I(talib.MA,self.data.Close,10)
        #self.ma20 = self.I(talib.MA,self.data.Close,20)
        self.MACD_DIF =  macdFast
        self.MACD_DIM = macdSlow
        self.MACD_HIS = histogram
        self.RSI = self.I(talib.RSI,self.data.Close,self.RSIperiod)
        self.ma5_1HR = resample_apply('1H',talib.MA,self.data.Close,5)
        self.ma10_1HR = resample_apply('1H',talib.MA,self.data.Close,10)
        #self.ma20_1HR = resample_apply('1H',talib.MA,self.data.Close,20)
        self.ma60_1HR = resample_apply('1H',talib.MA,self.data.Close,60)
        self.ma120_1HR = resample_apply('1H',talib.MA,self.data.Close,120)
        self.ema5_1HR = resample_apply('1H',talib.EMA,self.data.Close,5)
        self.ema10_1HR = resample_apply('1H',talib.EMA,self.data.Close,10)
        self.ema20_1HR = resample_apply('1H',talib.EMA,self.data.Close,20)
        self.ema60_1HR = resample_apply('1H',talib.EMA,self.data.Close,60)
        self.ema120_1HR = resample_apply('1H',talib.EMA,self.data.Close,120)
        
        
        # cross timeframe
        #self.emaD = resample_apply('D',talib.EMA,self.data.Close,10)
        #self.ema1HR = resample_apply('1H',talib.EMA,self.data.Close,10)
        #self.ema4HR = resample_apply('4H',talib.EMA,self.data.Close,10)
        
        upper,middle,lower= resample_apply('D',talib.BBANDS,self.data.Close,self.BollPeriod)
        self.boll_1D_up = upper
        self.boll_1D_mid = middle
        self.boll_1D_low = lower
       
    def log(self, show_sig = False):
         #position active
        if self.close_long_sig & self.long_sig :
            with open('log.txt','a') as logFile:
                    logFile.write(str(self.data.index[-1])+"****long conflict.\n")
        if self.close_short_sig & self.short_sig :
            with open('log.txt','a') as logFile:
                    logFile.write(str(self.data.index[-1])+"****short conflict.\n")
        if self.position:
            if self.close_long_sig & self.position.is_long:
                with open('log.txt','a') as logFile:
                    logFile.write(str(self.data.index[-1])+ "PL:"+str(self.position.pl_pct) + " close long .\n")
            elif self.close_short_sig & self.position.is_short:
                with open('log.txt','a') as logFile:
                    logFile.write(str(self.data.index[-1])+ "PL:"+str(self.position.pl_pct) + " close short .\n")
            # did nothing but get signal
            elif self.long_sig & show_sig:
                with open('log.txt','a') as logFile:
                    logFile.write(str(self.data.index[-1])+" ##long sig.\n")
            elif self.short_sig & show_sig:
                with open('log.txt','a') as logFil:
                    logFile.write(str(self.data.index[-1])+" ##short sig.\n")
            elif self.close_long_sig & show_sig:
                with open('log.txt','a') as logFile:
                    logFile.write(str(self.data.index[-1])+" ##close long sig.\n")
            elif self.close_short_sig & show_sig:
                with open('log.txt','a') as logFile:
                    logFile.write(str(self.data.index[-1])+" ##close short sig.\n")
        else:
            if self.long_sig:
                with open('log.txt','a') as logFile:
                    logFile.write(str(self.data.index[-1])+" long.\n")
            if self.short_sig:
                with open('log.txt','a') as logFile:
                    logFile.write(str(self.data.index[-1])+" short.\n")
            if self.close_long_sig & show_sig:
                with open('log.txt','a') as logFile:
                    logFile.write(str(self.data.index[-1])+" ##close long sig.\n")
            if self.close_short_sig & show_sig:
                with open('log.txt','a') as logFile:
                    logFile.write(str(self.data.index[-1])+" ##close short sig.\n")
    def dynLog(self):
        if self.close_long_sig & self.position.is_long:
            with open('log.txt','a') as logFile:
                    logFile.write(str(self.data.index[-1])+" close long\n")
        if self.close_short_sig & self.position.is_short:
            self.position.close()
        if self.long_sig:
             with open('log.txt','a') as logFile:
                    logFile.write(str(self.data.index[-1])+" long.\n")
        if self.short_sig:
            self.sell()
        ##sig
        if self.close_long_sig & (not self.position):
            with open('log.txt','a') as logFile:
                logFile.write(str(self.data.index[-1])+"## close longsig\n")
                
    def next(self):
        # split strategy for combine
        long_ema = (self.ema5[-1]>self.ema10[-1])&(self.ema10[-1]>self.ema20[-1])&(self.ema20[-1]>self.ema60[-1])&(self.ema60[-1]>self.ema200[-1])
        ema5_cross = (self.ema5[-2] < self.data.Close[-2]) & (self.ema5[-1] > self.data.Close[-1])
        long_macd = self.MACD_DIF[-1] > self.MACD_DIM[-1]
        long_ma = self.ma5[-1] > self.ma10[-1]
        long_1hr_ma = (self.ema5_1HR[-1]>self.ema10_1HR[-1])&(self.ema10_1HR[-1]>self.ema20_1HR[-1])&(self.ema20_1HR[-1]>self.ema60_1HR[-1])&(self.ema60_1HR[-1]>self.ema120_1HR[-1])
        
        short_rsi = self.RSI[-1] < self.RSICloseDownTrigger
        short_cross = self.data.Close[-1] < self.ema5_1HR[-1]
        short_ma_cross = self.ma5_1HR[-1] < self.ma60_1HR[-1]
        short_ema_cross = self.ema10_1HR[-1] < self.ema120_1HR[-1]
        short_ema = (self.ema5[-1]<self.ema10[-1])&(self.ema10[-1]<self.ema20[-1])&(self.ema20[-1]<self.ema60[-1])&(self.ema60[-1]<self.ema200[-1])
        
        short_boll = self.data.Close[-1] > self.boll_1D_up[-1]
        
        #combine signal
        self.long_sig = long_ema & ema5_cross & long_macd
        self.close_long_sig = (short_rsi & short_ema & short_cross & short_ma_cross)
        self.short_sig = False;
        self.close_short_sig = False;
        #position active
        
        #if self.position:
        #    #close long
        #    if self.close_long_sig & self.position.is_long:
        #        self.position.close()
        #    #close short
        #    elif self.close_short_sig & self.position.is_short:
        #        self.position.close()
        #no any position
        #else:
        #    if self.long_sig:
        #        self.buy()
        #    if self.short_sig:
        #        self.sell()
        if self.close_long_sig & self.position.is_long:
            self.position.close()
        if self.close_short_sig & self.position.is_short:
            self.position.close()
        if self.long_sig & (not self.close_long_sig):
            self.buy(size = self.dynSize/100)
        if self.short_sig:
            self.sell()
        self.log()
        #self.dynLog()
        
        #if(self.position.pl_pct > 0.3):
        #   self.position.close()
        #if(self.position.pl_pct < -0.05):
        #    self.position.close()
            
        super().next()

In [67]:
backtestData = df.rename(columns={"open": "Open", "high": "High", "low": "Low", "close": "Close","volume":"Volume"})
backtestData = backtestData[["Open","High","Low","Close","Volume"]]
result1 = Backtest(backtestData, StrategyMACD, cash=100000, commission=.002)
print(result1.run())
result1.plot()

Start                     2019-11-01 00:00:00
End                       2020-11-28 08:15:00
Duration                    393 days 08:15:00
Exposure Time [%]                     64.7561
Equity Final [$]                       324362
Equity Peak [$]                        368476
Return [%]                            224.362
Buy & Hold Return [%]                 85.6685
Max. Drawdown [%]                    -22.5944
Avg. Drawdown [%]                     -1.6688
Max. Drawdown Duration       80 days 21:00:00
Avg. Drawdown Duration        1 days 17:17:00
# Trades                                   19
Win Rate [%]                              100
Best Trade [%]                        59.7532
Worst Trade [%]                      0.396257
Avg. Trade [%]                        21.8023
Max. Trade Duration          93 days 13:00:00
Avg. Trade Duration          61 days 20:45:00
Profit Factor                             NaN
Expectancy [%]                            NaN
SQN                               

C:\ProgramData\Anaconda3\lib\site-packages\backtesting\_plotting.py:104: UserWarning: Data contains too many candlesticks to plot; downsampling to '1H'. See `Backtest.plot(resample=...)`
  warnings.warn("Data contains too many candlesticks to plot; downsampling to {!r}. "


In [66]:
stats._strategy


<Strategy StrategyMACD(dynSize=10)>

In [69]:
#optimize
stats = result1.optimize(dynSize=range(10, 100, 5),
                    maximize='-Max. Drawdown [%]')
stats

ValueError: `maximize`, if str, must match a key in pd.Series result of backtest.run()